In [2]:
import pandas as pd
import numpy as np
import re

In [96]:
def get_data(day:int)->list:

    data = []
    f = open(f'data/data{day}.txt', 'r')
    for line in f:
        data.append(line)
    data = [i.replace('\n','') for i in data]
    return data


# DAY 1

In [295]:
#1
data = [int(i) for i in get_data(day=1)]

def compare_if_increased(data:list)->list:
    '''
    if the value in a list is higher than the prev value then return inc, else return idk
    '''
    res = []
    for i in range(len(data)-1):
        if data[i] < data[i+1]:
            res.append('inc')
        else:
            res.append('idk')
            
    return res

res = compare_if_increased(data)
res.count('inc')

1226

In [296]:
#2
data3 = pd.Series(data).rolling(3).sum().dropna().values
res = compare_if_increased(data3)
res.count('inc')

1252

# DAY 2

In [24]:
#1
data=get_data(day=2)

def get_int_from_string(string):
    i = ''.join(x for x in string if x.isdigit())
    return int(i)
    
fw = [get_int_from_string(i) for i in data if 'forward' in i]
up = [get_int_from_string(i) for i in data if 'up' in i]
dn = [get_int_from_string(i) for i in data if 'down' in i]

(sum(dn) - sum(up)) * sum(fw)

1804520

In [25]:
#2
h = 0
aim = 0
d = 0

for i in data:
    if 'forward' in i:
        i = get_int_from_string(i)
        h += i
        d += i*aim
    elif 'down' in i:   

        aim += get_int_from_string(i)
    elif 'up' in i:    
        aim -= get_int_from_string(i)
h * d

1971095320

# DAY 3

In [109]:
#1
data = get_data(day=3)
df = pd.DataFrame([[i for i in char] for char in data])

mode = df.mode().values[0]
gr_bit = ''.join(mode)
er_bit = ''.join(['1' if i =='0' else '0' for i in mode])

int(er_bit,2) * int(gr_bit,2)


1307354

In [110]:
#2
def get_rating(data, method):
    keep = data.copy()
    n = len(keep)
    for i in range(12):
        k_ = keep.mode()[i].fillna('-1').max()
        if method =='mode':
            k = k_
        else:
            k = str(abs(int(k_)-1))
        keep = keep.loc[keep[i] == k, :]
        n = len(keep)
        if n ==1: break
    return ''.join(keep.values[0])

    
o2 = get_rating(df, method='mode')
co2 = get_rating(df, method='notmode')

int(o2,2) * int(co2,2)

482500

# DAY 4

In [299]:
#1
data = get_data(day=4)

def get_bingo_nums(data):
    nums = data.pop(0).split(',')
    while True: 
        try:
            data.remove('')
        except: break
    return nums, data

def get_bingo_boards(board_data):
    boards = {}
    for i in range(100):
        l = []
        for _ in range(5):
            r = board_data.pop(0).split(' ')
            while True: 
                try:
                    r.remove('')
                except: break
            l.append(r)
        boards[str(i)] = {'board' : pd.DataFrame(l)}
    return boards

def make_bingo(data):
    
    nums, board_data = get_bingo_nums(data)
    boards = get_bingo_boards(board_data)
    
    return nums, boards

def run_bingo(nums, boards):
    winner = []
    numbers_called = []
    for n in nums:
        numbers_called.append(n)
        for i in boards.keys():
            # check if any rows have all values:
            for row in boards[i]['board'].values:
                if len(set(row).intersection(numbers_called)) == 5:
                    winner = boards[i]['board']
                    print(f'BOARD {i} ROW {row } WINS AFTER {len(numbers_called)} NUMBERS')
                    return winner, numbers_called
                    break
            #check if any columns have all values
            for col in boards[i]['board']:
                if len(set(boards[i]['board'][col]).intersection(numbers_called)) == 5:
                    winner = boards[i]['board']
                    print(f'BOARD {i} COLUMN {col} WINS AFTER {len(numbers_called)} NUMBERS')
                    return winner, numbers_called
                    break

def get_bingo_score(winner, numbers_called):
    winner_flat = [int(i) for i in winner.values.flatten()]
    marked = set(winner_flat).intersection(set([int(i) for i in numbers_called]))
    return (sum(winner_flat)-sum(marked)) * int(numbers_called[-1])

nums, boards = make_bingo(data)
winner, numbers_called = run_bingo(nums, boards)
get_bingo_score(winner, numbers_called)

BOARD 11 ROW ['49' '69' '76' '98' '22'] WINS AFTER 22 NUMBERS


23177

In [307]:
#2
def run_bingo_loser(nums, boards):
    winner = []
    winners=[]
    numbers_called = []
    for n in nums:
        numbers_called.append(n)
        for i in boards.keys():
            # check if any rows have all values:
            for row in boards[i]['board'].values:
                if len(set(row).intersection(numbers_called)) == 5:
                    winner = boards[i]['board']
                    winners.append(i)
            #check if any columns have all values
            for col in boards[i]['board']:
                if len(set(boards[i]['board'][col]).intersection(numbers_called)) == 5:
                    winner = boards[i]['board']
                    winners.append(i)
            if len(set(winners)) == 100:
                print(f'BOARD {i} ROW {row } LOSES AFTER {len(numbers_called)} NUMBERS')
                return winner, numbers_called, winners
                
loser, numbers_called, winners = run_bingo_loser(nums, boards)
get_bingo_score(loser, numbers_called)

BOARD 59 ROW ['67' '70' '14' '77' '86'] LOSES AFTER 87 NUMBERS


6804

# DAY 5

In [157]:
#1
data = get_data(day=5)

cols = ['x1','y1','x2','y2']
df = pd.DataFrame(data=[re.split(' -> |,',i) for i in data], columns=cols).astype(int)


def get_grid(n):
    grid = [[0 for x in range(n)] for i in range(n)]
    return grid

def get_horizontal_vertical_only(df):
    #only consider horizontal or vertical movements
    dfy = df.loc[(df.x1 == df.x2), :].copy()
    dfx = df.loc[(df.y1 == df.y2), :].copy()
    return {'x': dfx, 'y': dfy}

def populate_grid(df):
    dfs = get_horizontal_vertical_only(df)
    grid = get_grid(1000)
    
    for key in dfs.keys():
        for row in range(len(dfs[key])):
            line = dfs[key].iloc[row]
            start = min(line[f'{key}1'], line[f'{key}2'])
            stop = max(line[f'{key}1'], line[f'{key}2'])
            
            if key == 'x':
                p = line['y1'] 
                for i in range(start, stop+1):
                    grid[p][i] += 1
            if key == 'y':
                p = line['x1']
                for i in range(start,stop+1):
                    grid[i][p] += 1           
    return grid

grid = populate_grid(df)
np.count_nonzero(np.array(grid) >= 2)

7438

In [158]:
#2
def get_diag(df):
    #only consider horizontal or vertical movements
    return  df.loc[(df.x1 != df.x2) & (df.y1 != df.y2) , :].copy()

def populate_grid_diag(df, grid):
    dfd = get_diag(df)
    for row in range(len(dfd)):
        line = dfd.iloc[row]

        vals = {'x':None, 'y':None}
        for key in vals.keys():
            if line[key+'1'] < line[key+'2']:
                vals[key] = range(line[key+'1'], line[key+'2'] +1)
            else: 
                vals[key] = range(line[key+'1'], line[key+'2']-1, -1)
        
        for n in range(len(vals['x'])):
            grid[vals['y'][n]][vals['x'][n]] +=1

    return grid


grid_diag = populate_grid_diag(df, grid)
np.count_nonzero(np.array(grid_diag) >= 2)

21406

# DAY 6

In [159]:
#1
data = get_data(day=6)


